# Programación declarativa @ URJC
# Programación funcional
## Examen Convocatoria Ordinaria (3 de febrero de 2021)
## Curso 20-21

# Definiciones auxiliares

In [1]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


### Algunas funciones sobre tipos estándar de la librería de Scala

In [2]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
        
        // Reverse a list
        // e.g. List(1,2,3).reverse==List(3,2,1)
        def reverse: List[A]
        
        // Take the first `n` elements of the list
        // e.g. List(1,2,3).take(2) == List(1,2)
        //      List(1,2,3).take(0) == List()
        //      List(1,2,3).take(5) == List(1,2,3)
        def take(n: Int): List[A]
        
        // Drop the first `n` elements of the list 
        // e.g. List(1,2,3).drop(2) == List(3)
        //      List(1,2,3).drop(0) == List(1,2,3)
        //      List(1,2,3).drop(4) == List()
        def drop(n: Int): List[A]

        // List concatenation
        // e.g. List(1,2,3).concat(List(4,5)) == List(1,2,3,4,5)
        def concat(l: List[A]): List[A]
    }
    
    abstract class Option[A]{
        // Test whether the value is defined (i.e. `Some`) or not (i.e. `None`)
        def isDefined: Boolean 
        def map[B](f: A => B): Option[B]
    }
    
    abstract class Either[A, B]{
        // Test whether the value is left or right
        def isLeft: Boolean 
        def isRight: Boolean 
        def map[C](f: B => C): Either[A, C]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard

In [3]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)
type Or[P, Q] = Either[P, Q]
type And[P, Q] = (P, Q)

defined type Not
defined type <=>
defined type Or
defined type And

# Ejercicio 1 (variante 1)


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar que los siguientes razonamientos de la lógica proposicional representan deducciones válidas de la lógica intuicionista: 

* Dilema constructivo complejo: $\{p ∨ q, p→r, q→s\} ⊢ r ∨ s$

In [5]:
def proof[P,Q,R,S](p1: Either[P,Q], p2: P => R, p3: Q => S): Either[R,S] =
    p1 match
        case Left(p : P) => Left(p2(p) : R) : Either[R,S]
        case Right(q: Q) => Right(p3(q): S) : Either[R,S]

defined function proof

In [ ]:
def proof[P, Q, R, S](p1: Either[P, Q], p2: P => R, p3: Q => S): Either[R, S] = 
    p1 match {
        case Left(p: P) => Left(p2(p))
        case Right(q: Q) => Right(p3(q))
    }

* Dilema destructivo simple: $\{¬p ∨¬q, r→p, r→q\} ⊢ ¬r$

In [7]:
def proof[P,Q,R](p1: Either[Not[P],Not[Q]], p2: R => P, p3: R => Q): (R => Nothing) =
    (r: R) => 
        ??? : Nothing

defined function proof

In [8]:
def proof[P,Q,R](p1: Either[Not[P],Not[Q]], p2: R => P, p3: R => Q): (R => Nothing) =
    (r: R) => 
        p1 match
            case Left(np: Not[P]) => np(p2(r)) : Nothing
            case Right(nq: Not[Q]) => nq(p3(r)) : Nothing
        

defined function proof

In [ ]:
def proof[P, Q, R](p1: Either[Not[P], Not[Q]], p2: R => P, p3: R => Q): Not[R] = 
    (r: R) => p1 match {
        case Left(np: Not[P]) => np(p2(r))
        case Right(nq: Not[Q]) => nq(p3(r))
    }

* Dilema destructivo complejo: $\{¬p ∨¬q, r→p, s→q\} ⊢ ¬r ∨ ¬s$

In [ ]:
def proof[P,Q,R,S](p1: Either[Not[P], Not[Q]], p2: R => P, p3: S => Q): Either[Not[R], Not[S]] =
    p1 match 
        case Left(np) => Left(??? : (R => Nothing))
        case Right(nq) => Right(??? : (S => Nothing))

In [9]:
def proof[P,Q,R,S](p1: Either[Not[P], Not[Q]], p2: R => P, p3: S => Q): Either[Not[R], Not[S]] =
    p1 match 
        case Left(np) => Left((r: R) => np(p2(r)) : Nothing)
        case Right(nq) => Right((s: S) => nq(p3(s)) : Nothing)

defined function proof

In [16]:
def suma1(n: Int): Int = n+1

val sum1: Int => Int = n => n+1

defined function suma1
sum1: Int => Int = ammonite.$sess.cmd15$Helper$$Lambda$2685/0x00000008016c9040@f00c19

In [19]:

def proof[P, Q, R, S](p1: Either[Not[P], Not[Q]], p2: R => P, p3: S => Q): Either[Not[R], Not[S]] = 
    p1 match {
        case Left(np: Not[P]) => Left((r: R ) => 
            np(p2(r)))
        case Right(nq: Not[Q]) => Right((s: S) => nq(p3(s)))
    }

def proof[P, Q, R, S]: Either[Not[P], Not[Q]] => (R => P) => (S => Q) => Either[Not[R], Not[S]] =
    p1 => p2 => p3 => p1 match {
        case Left(np: Not[P]) => Left((r: R) => np(p2(r)))
        case Right(nq: Not[Q]) => Right((s: S) => nq(p3(s)))
    }


defined function proof
defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$⊢ ((p →q) →p)→p$

Supóngase para ello que la ley del tercio excluso se cumple para la variable proposicional $p$, es decir, que la fórmula $p ∨ ¬p$  puede utilizarse como premisa.


In [ ]:
def proof[P,Q](p1: Either[P,Not[P]]): (((P => Q) => P) => P) =
    ??? : (((P => Q) => P) => P)

In [ ]:
def proof[P,Q](p1: Either[P,Not[P]]): (((P => Q) => P) => P) =
    (pqp : ((P => Q) => P)) =>
        ??? : P

In [10]:
def proof[P,Q](p1: Either[P,Not[P]]): (((P => Q) => P) => P) =
    (pqp : ((P => Q) => P)) =>
        p1 match 
            case Left(p) => p
            case Right(np) => pqp(np)

defined function proof

In [ ]:
def proof[P, Q](pnp: Either[P, Not[P]]): ((P => Q) => P) => P = 
    (f: (P => Q) => P) => 
        pnp match {
            case Left(p) => p
            case Right(np) =>
                // f((p: P) => np(p) : Q) 
                f(np)
        }

# Ejercicio 1 (variante 2)


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar que los siguientes teoremas de la lógica intuicionista: 

* Dilema constructivo complejo: $⊢ (p ∨ q) → (p→r) → (q→s) → (r ∨ s)$

In [ ]:
def proof[P, Q, R, S]: Either[P, Q] => (P => R) => (Q => S) => Either[R, S] = 
    p1 => p2 => p3 => p1 match {
        case Left(p: P) => Left(p2(p))
        case Right(q: Q) => Right(p3(q))
    }

* Dilema destructivo simple: $⊢ (¬p ∨¬q)→(r→p)→(r→q)→¬r$

In [ ]:
def proof[P, Q, R]: Either[Not[P], Not[Q]] => (R => P) => (R => Q) => Not[R] = 
    p1 => p2 => p3 => (r: R) => p1 match {
        case Left(np: Not[P]) => np(p2(r))
        case Right(nq: Not[Q]) => nq(p3(r))
    }

* Dilema destructivo complejo: $⊢(¬p ∨¬q)→(r→p)→(s→q) → (¬r ∨ ¬s)$

In [ ]:
def proof[P, Q, R, S]: Either[Not[P], Not[Q]] => (R => P) => (S => Q) => Either[Not[R], Not[S]] = 
    p1 => p2 => p3 => p1 match {
        case Left(np: Not[P]) => Left((r: R) => np(p2(r)))
        case Right(nq: Not[Q]) => Right((s: S) => nq(p3(s)))
    }

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$⊢ ((p →q) →p)→p$

Supóngase para ello que la ley de la doble negación se cumple para la variable proposicional $p$, es decir, que la fórmula  $¬¬p→p$  puede utilizarse como premisa.

In [ ]:
def proof[P, Q]: ((Not[Not[P]] => Not[Not[Q]]) => Not[Not[P]]) => Not[Not[P]] = 
    (f: (Not[Not[P]] => Not[Not[Q]]) => Not[Not[P]]) => 
        (np: Not[P]) => 
            f((nnp : Not[Not[P]]) => nnp(np) : Not[Not[Q]])(np) : Nothing

In [ ]:
def proof[P, Q](dn: Not[Not[P]] => P): ((P => Q) => P) => P = 
    (f: (P => Q) => P) => 
        dn((np: Not[P]) => 
           // np(f((p: P) => np(p) : Q)) : Nothing)
           np(f(np)))

# Ejercicio 2 
__(1 punto)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(X+1)^2 \cong X^2+2*X+1$

de tal forma que se verifique el siguiente test unitario para $X=Boolean$:

_(variante 1)_

In [ ]:
def from[X](a: (Option[X], Option[X])): Either[(X,X), Either[X, Option[X]]] =
    ??? : Either[(X,X), Either[X, Option[X]]]

In [21]:
def from[X](a: (Option[X], Option[X])): Either[(X,X), Either[X, Option[X]]] =
    a match 
        case (None, None) => Right(Right(None)) : Either[(X,X), Either[X, Option[X]]]
        case (Some(x1), Some(x2)) => Left((x1,x2)) : Either[(X,X), Either[X, Option[X]]]
        case (Some(x),None) => Right(Left(x)) : Either[(X,X), Either[X, Option[X]]]
        case (None, Some(x)) =>  Right(Right(Some(x))) : Either[(X,X), Either[X, Option[X]]]

defined function from

In [16]:
def to[X](b: Either[(X,X), Either[X, Option[X]]]): (Option[X], Option[X]) = 
    ??? : (Option[X], Option[X])

defined function b

In [16]:
def to[X](b: Either[(X,X), Either[X, Option[X]]]): (Option[X], Option[X]) = 
    (??? : Option[X], ??? : Option[X])

defined function b

In [23]:
def to[X](b: Either[(X,X), Either[X, Option[X]]]): (Option[X], Option[X]) = 
    b match 
        case Left((x1,x2)) => (Some(x1),Some(x2))
        case Right(Left(x)) => (Some(x),None)
        case Right(Right(None)) => (None, None)
        case Right(Right(Some(x))) => (None, Some(x))

defined function to

In [18]:
class IsoTest(
    from: ((Option[Boolean], Option[Boolean])) => Either[(Boolean, Boolean), Either[Boolean, Option[Boolean]]],
    to: Either[(Boolean, Boolean), Either[Boolean, Option[Boolean]]] => (Option[Boolean], Option[Boolean])
) extends AnyFlatSpec with should.Matchers{
    
    "from-to" should "work" in {
        from(to(Left((true,true)))) shouldBe Left((true, true))
        from(to(Right(Left(true)))) shouldBe Right(Left(true))
        from(to(Right(Right(Some(true))))) shouldBe Right(Right(Some(true)))
        from(to(Right(Right(None)))) shouldBe Right(Right(None))
    }
    
    "to-from" should "work" in {
        to(from((None, None))) shouldBe (None, None)
        to(from((Some(false), None))) shouldBe (Some(false), None)
        to(from((None, Some(true)))) shouldBe (None, Some(true))
        to(from((Some(true), Some(false)))) shouldBe (Some(true), Some(false))
    }
}

defined class IsoTest

In [ ]:
def from[X](l: (Option[X], Option[X])): Either[(X, X), Either[X, Option[X]]] = 
        l match {
            case (Some(a1), Some(a2)) => Left((a1,a2))
            case (Some(a1), None) => Right(Left(a1))
            case (None, maybeA2) => Right(Right(maybeA2))
        }


In [ ]:
def to[X](l: Either[(X, X), Either[X, Option[X]]]): (Option[X], Option[X]) = 
        l match {
            case Left((a1,a2)) => (Some(a1), Some(a2))
            case Right(Left(a1)) => (Some(a1), None)
            case Right(Right(maybeA2)) => (None, maybeA2)
        }

In [24]:
run(new IsoTest(from, to))

cell18$Helper$IsoTest:
from-to
- should work
to-from
- should work


_(variante 2)_

In [ ]:
class IsoTest(
    from: ((Either[Boolean, Unit], Either[Boolean, Unit])) => 
                Either[(Boolean, Boolean), Either[Boolean, Either[Boolean, Unit]]],
    to: Either[(Boolean, Boolean), Either[Boolean, Either[Boolean, Unit]]] => 
                (Either[Boolean, Unit], Either[Boolean, Unit])
) extends FlatSpec with Matchers{
    
    "from-to" should "work" in {
        from(to(Left((true, true)))) shouldBe Left((true, true))
        from(to(Right(Left(true)))) shouldBe Right(Left(true))
        from(to(Right(Right(Left(true))))) shouldBe Right(Right(Left(true)))
        from(to(Right(Right(Right(()))))) shouldBe Right(Right(Right(())))
    }
    
    "to-from" should "work" in {
        to(from((Right(()), Right(())))) shouldBe (Right(()), Right(()))
        to(from((Left(false), Right(())))) shouldBe (Left(false), Right(()))
        to(from((Right(()), Left(true)))) shouldBe (Right(()), Left(true))
        to(from((Left(true), Left(false)))) shouldBe (Left(true), Left(false))
    }
}

In [ ]:
def from[X](l: (Either[X, Unit], Either[X, Unit])): Either[(X, X), Either[X, Either[X, Unit]]] = 
        l match {
            case (Left(a1), Left(a2)) => Left((a1,a2))
            case (Left(a1), _) => Right(Left(a1))
            case (_, maybeA2) => Right(Right(maybeA2))
        }


In [ ]:
def to[X](l: Either[(X, X), Either[X, Either[X, Unit]]]): (Either[X, Unit], Either[X, Unit]) = 
        l match {
            case Left((a1,a2)) => (Left(a1), Left(a2))
            case Right(Left(a1)) => (Left(a1), Right(()))
            case Right(Right(maybeA2)) => (Right(()), maybeA2)
        }

In [ ]:
run(new IsoTest(from, to))

# Ejercicio 3 (variante 1)
__(3 puntos)__

La función de orden superior `sequence` recibe una lista de valores opcionales y devuelve una lista con todos los valores pertenecientes a la lista de entrada (en el mismo orden), _en caso de que todos los valores de la lista de entrada estén definidos_; si alguno de los valores opcionales de la lista de entrada es `None`, entonces la función sequence devuelve `None` también. El comportamiento de la función se ilustra en el siguiente test unitario:


In [26]:
class SequenceTest(
    sequence: List[Option[Int]] => Option[List[Int]]
) extends AnyFlatSpec with should.Matchers{
    "sequence" should "work" in {
        sequence(List(Some(1), Some(2), Some(3))) shouldBe Some(List(1,2,3))
        sequence(List(None, Some(2), Some(3))) shouldBe None
        sequence(List(Some(1), None, Some(3))) shouldBe None
        sequence(List(Some(5))) shouldBe Some(List(5))
        sequence(List(None)) shouldBe None
        sequence(List()) shouldBe Some(List())
    }
}

defined class SequenceTest

__a) (1 punto)__ Implementa la función `sequence` de manera recursiva.

In [30]:
def sequence[A](l: List[Option[A]]): Option[List[A]] =
    l match 
        case Nil => Some(Nil)
        case None :: tail => None
        case Some(x) :: tail => 
            val tailSol: Option[List[A]] = sequence(tail)
            tailSol match
                case None => None
                case Some(list) => Some(x :: list)
                        

defined function sequence

In [5]:
def sequence[A](l: List[Option[A]]): Option[List[A]] = 
    l match {
        case Nil => Some(Nil)
        case None :: _ => None
        case Some(h) :: tail => 
            sequence(tail) match {
                case None => None
                case Some(t: List[A]) => Some(h :: t)
            }
    }

defined function sequence

In [5]:
def sequence[A](l: List[Option[A]]): Option[List[A]] = 
    l match {
        case Nil => Some(Nil)
        case None :: _ => None
        case Some(h) :: tail => 
            sequence(tail) match {
                case None => None
                case Some(t: List[A]) => Some(h :: t)
            }
    }

defined function sequence

In [31]:
run(new SequenceTest(sequence))

cell26$Helper$SequenceTest:
sequence
- should work


__b) (1 punto)__ Implementa la función `sequence` utilizando __`foldRight`__.

In [34]:
// def sequence[A](l: List[Option[A]]): Option[List[A]] =
//     l match 
//         case Nil => Some(Nil)
//         case None :: tail => None
//         case Some(x) :: tail => 
//             val tailSol: Option[List[A]] = sequence(tail)
//             tailSol match
//                 case None => None
//                 case Some(list) => Some(x :: list)

def sequenceFR[A](l: List[Option[A]]): Option[List[A]] =
    l.foldRight(Some(Nil))((head: Option[A], tailSol: Option[List[A]]) => 
                              (head,tailSol) match
                                   case (None,_) => None 
                                   case (_, None) => None
                                   case (Some(x), Some(list)) => Some(x :: list)
                          )
                        

defined function sequenceFR

In [ ]:
def sequenceFR[A](l: List[Option[A]]): Option[List[A]] = 
    l.foldRight[Option[List[A]]](Some(Nil)){
        case (Some(h), Some(t)) => Some(h::t)
        case _ => None
    }


In [9]:
def sequenceFR[A](l: List[Option[A]]): Option[List[A]] = 
    l.foldRight[Option[List[A]]](
        Some(Nil))( // solución para el caso base
        (head: Option[A], tailSol: Option[List[A]]) =>  // solución para el caso recursivo
            head match {
                case None => 
                    tailSol match {
                        case None => None : Option[List[A]]
                        case Some(l) => None : Option[List[A]]
                    }
                case Some(a) => 
                    tailSol match {
                        case None => None : Option[List[A]]
                        case Some(l) => 
                            Some(a :: l) : Option[List[A]]
                    }
            }
    )


defined function sequenceFR

In [10]:
def sequenceFR[A](l: List[Option[A]]): Option[List[A]] = 
    l.foldRight[Option[List[A]]](
        Some(Nil))( // solución para el caso base
        (head: Option[A], tailSol: Option[List[A]]) =>  // solución para el caso recursivo
            (head, tailSol) match {
                case (None, None) => None : Option[List[A]]
                case (None, Some(l)) => None : Option[List[A]]
                case (Some(a), None) => None : Option[List[A]]
                case (Some(a), Some(l)) => Some(a :: l) : Option[List[A]]
            }
    )


defined function sequenceFR

In [12]:
def sequenceFR[A](l: List[Option[A]]): Option[List[A]] = 
    l.foldRight[Option[List[A]]](
        Some(Nil))( 
        {
            case (Some(a), Some(l)) => Some(a :: l) : Option[List[A]]
            case _ => None : Option[List[A]]
        }
    )


defined function sequenceFR

In [15]:
def sequence[A](l: List[Option[A]]): Option[List[A]] = 
    l match {
        case Nil => Some(Nil)
        case (head: Option[A]) :: tail =>
            val tailSol: Option[List[A]] = sequence(tail)
            (head, tailSol) match {
                case (Some(a), Some(l)) => Some(a :: l) : Option[List[A]]
                case _ => None
            }
    }

defined function sequence

In [10]:
def sequenceFR[A](l: List[Option[A]]): Option[List[A]] = 
    l.foldRight[Option[List[A]]](
        Some(Nil))( // solución para el caso base
        (head: Option[A], tailSol: Option[List[A]]) =>  // solución para el caso recursivo
            (head, tailSol) match {
                case (Some(a), Some(l)) => Some(a :: l) : Option[List[A]]
                case _ => None   
            }
    )


defined function sequenceFR

In [35]:
run(new SequenceTest(sequenceFR))

cell26$Helper$SequenceTest:
sequence
- should work


__c) (1 punto)__ Se desea implementar una función que reciba una lista de enteros, divida todos sus elementos entre un valor dado y, finalmente, multiplique los resultados de las divisiones, _siempre y cuando todas las divisiones hayan resultado en un valor entero_. En caso de que la división de algunos de los elementos no haya sido entera o no se haya podido realizar (en el caso de la división por cero), la función no devolverá ningún número. Por ejemplo:



In [38]:
class TestOp(op: (List[Int], Int) => Option[Int]) extends AnyFlatSpec with should.Matchers{
    "op" should "work" in {
        op(List(2,4,6), 2) shouldBe Some(2/2*4/2*6/2)
        op(List(3,6,9), 3) shouldBe Some(6)
        op(List(3,5,9), 3) shouldBe None
        op(List(), 5) shouldBe Some(1)
        op(List(), 0) shouldBe Some(1)
        op(List(2,5,2), 0) shouldBe None
    }
}

defined class TestOp

Implementa la función `op` utilizando `sequence` y otras funciones de orden superior del catálogo explicado en clase.

In [42]:
5 / 3

res42: Int = 1

In [50]:
def op(l: List[Int], v: Int): Option[Int] =
    val option: Option[List[Int]] = sequenceFR(l.map(e => 
            v match 
                  case 0 => None
                  case v if e % v != 0 => None
                  case _ => Some(e / v : Int)))
    option match
        case None => None
        case Some(lista: List[Int]) => Some(lista.foldRight(1)(_ * _))

defined function op

In [ ]:
def op(l: List[Int], d: Int): Option[Int] = 
    sequence(
        l.map(n => 
              if (d != 0 && n % d == 0) Some(n / d) 
              else None
        )
    ).map(_.foldLeft(1)(_*_))

In [49]:
// También se admite como válida la solución sin el uso de `Option::map`
def op(l: List[Int], d: Int): Option[Int] = 
    sequence(
        l.map(n => 
              if (d != 0 && n % d == 0) Some(n / d) 
              else None
        )
    ) match {
        case Some(l) => Some(l.foldLeft(1)(_*_))
        case None => None
    }

defined function op

In [48]:
run(new TestOp(op))

cell38$Helper$TestOp:
op
- should work


# Ejercicio 3 (variante 2)
__(3 puntos)__

La función de orden superior `sequence` recibe una lista de valores de tipo `X` o `Y` y devuelve una lista con los valores de tipo `Y` pertenecientes a la lista de entrada (en el mismo orden), _en caso de que dicha lista no contenga ningún valor de tipo `X`_; en caso de que sí lo contenga, la función `sequence` devolverá el primer valor de tipo `X` encontrado. El comportamiento de la función se ilustra en el siguiente test unitario, donde la función `sequence` se encuentra particularizada para los tipos `X=String` e `Y=Int`:


In [ ]:
class SequenceTest(
    sequence: List[Either[String, Int]] => Either[String, List[Int]]
) extends FlatSpec with Matchers{
    "sequence" should "work" in {
        sequence(List(Right(1), Right(2), Right(3))) shouldBe Right(List(1,2,3))
        sequence(List(Left("error1"), Right(2), Right(3))) shouldBe Left("error1")
        sequence(List(Right(1), Left("error1"), Right(3))) shouldBe Left("error1")
        sequence(List(Right(1), Left("error1"), Left("error2"))) shouldBe Left("error1")
        sequence(List(Right(5))) shouldBe Right(List(5))
        sequence(List(Left("error1"))) shouldBe Left("error1")
        sequence(List()) shouldBe Right(List())
    }
}

__a) (1 punto)__ Implementa la función `sequence` de manera recursiva.

In [ ]:
def sequence[A,B](l: List[Either[A, B]]): Either[A, List[B]] = 
    l match {
        case Nil => Right(Nil)
        case Left(a) :: _ => Left(a)
        case Right(h) :: tail => 
            sequence(tail) match {
                case Left(a) => Left(a)
                case Right(t) => Right(h :: t)
            }
    }

In [ ]:
run(new SequenceTest(sequence))

__b) (1 punto)__ Implementa la función `sequence` utilizando __`foldRight`__.

In [ ]:
def sequenceFR[A, B](l: List[Either[A, B]]): Either[A, List[B]] = 
    l.foldRight[Either[A, List[B]]](Right(Nil)){
        case (Right(h), Right(t)) => Right(h::t)
        case (Left(a), _) => Left(a)
        case (_, Left(b)) => Left(b)
    }


In [ ]:
run(new SequenceTest(sequenceFR))

__c) (1 punto)__ Se desea implementar una función que reciba una lista de enteros, divida todos sus elementos entre un valor dado y, finalmente, multiplique los resultados de las divisiones, _siempre y cuando todas las divisiones hayan resultado en un valor entero_. En caso de que la división de algunos de los elementos no haya sido entera o no se haya podido realizar (en el caso de la división por cero), la función devolverá la cadena `"error"`. Por ejemplo:



In [ ]:
class TestOp(op: (List[Int], Int) => Either[String, Int]) extends FlatSpec with Matchers{
    "op" should "work" in {
        op(List(2,4,6), 2) shouldBe Right(2/2*4/2*6/2)
        op(List(3,6,9), 3) shouldBe Right(6)
        op(List(3,5,9), 3) shouldBe Left("error")
        op(List(), 5) shouldBe Right(1)
        op(List(), 0) shouldBe Right(1)
        op(List(2,5,2), 0) shouldBe Left("error")
    }
}

Implementa la función `op` utilizando `sequence` y otras funciones de orden superior del catálogo explicado en clase.

In [ ]:
def op(l: List[Int], d: Int): Either[String, Int] = 
    sequence(
        l.map(n => 
              if (d != 0 && n % d == 0) Right(n / d) 
              else Left("error")
        )
    ).map(_.foldLeft(1)(_*_))

In [ ]:
def op(l: List[Int], d: Int): Either[String, Int] = 
    sequence(
        l.map(n => 
              if (d != 0 && n % d == 0) Right(n / d) 
              else Left("error")
        )
    ) match {
        case Left(error) => Left(error)
        case Right(l) => Right(l.foldLeft(1)(_*_))
    }

In [ ]:
run(new TestOp(op))

# Ejercicio 4
__(3 puntos)__

Considérese la siguiente implementación mediante recursión final de la función de orden superior `foldLeft`: 
```scala
def foldLeft[A, B](l: List[A])(acc: B)(f: (B, A) => B): B = 
    l match {
        case Nil => acc
        case h :: t => foldLeft(t)(f(acc, h))(f)
    }
```

__a) (1,5 puntos)__ Implementa una variante de la función `foldLeft`, denominada `foldLeftUntil`, que permita finalizar de manera anticipada la iteración cuando el valor acumulado hasta el momento satisfaga determinada condición. En caso de que el valor acumulado no cumpla nunca la condición, el comportamiento será el mismo de la función `foldLeft`.

In [51]:
def foldLeftUntil[A, B](l: List[A])(acc: B)(cond: B => Boolean)(f: (B, A) => B): B = 
    l match {
        case Nil => acc
        case h :: t if cond(acc) => acc
        case h :: t => foldLeftUntil(t)(f(acc, h))(cond)(f)
    }

defined function foldLeftUntil

In [ ]:
def foldLeftUntil[A, B](l: List[A])(acc: B)(cond: B => Boolean)(f: (B, A) => B): B =
    l match {
        case h :: t if !cond(acc) => 
            foldLeftUntil(t)(f(acc, h))(cond)(f)
        case _ => acc
    }

_(variante 1)_

__b) (1,5 puntos)__ Dada una lista de pares clave/valor, se desea implementar una función `lookup` que devuelva el valor asociado a una clave en caso de que exista. Si existen varias ocurrencias con la misma clave se devolverá el valor de la primera. Por ejemplo:

In [57]:
def lookup[A,B](l: List[(A,B)])(k: A): Option[B] =
    foldLeftUntil[(A,B),Option[B]](l)(None)(acc => acc match {case None => false case _ => true})((acc: Option[B], elem: (A,B)) => if elem._1 == k then Some(elem._2) else None)

defined function lookup

In [53]:
class TestLookup(
    lookup: List[(Int, String)] => Int => Option[String]
) extends AnyFlatSpec with should.Matchers{
    
    "lookup" should "work" in {
        lookup(List())(1) shouldBe None
        lookup(List((1, "a"), (2, "b"), (3, "c")))(1) shouldBe Some("a")
        lookup(List((1, "a"), (2, "b"), (1, "a_bis"), (3, "c")))(1) shouldBe Some("a")
        lookup(List((1, "a"), (2, "b"), (3, "c")))(2) shouldBe Some("b")
        lookup(List((1, "a"), (2, "b"), (3, "c")))(3) shouldBe Some("c")
        lookup(List((1, "a"), (2, "b"), (3, "c")))(0) shouldBe None
    }
}

defined class TestLookup

Implementa la función `lookup` utilizando la función `foldLeftUntil` del apartado anterior: 

In [ ]:
def lookup[A, B](list: List[(A, B)])(a: A): Option[B] =
    foldLeftUntil(list)(Option.empty[B])(_.isDefined){
        case (_, (`a`, b)) => Some(b) 
        case (acc, _) => acc 
    }

In [58]:
run(new TestLookup(lookup))

cell53$Helper$TestLookup:
lookup
- should work


_(variante 2)_

__b) (1,5 puntos)__ Dada una lista de pares clave/valor, se desea implementar una función `lookup` que devuelva el valor asociado a una clave en caso de que exista. Si existen varias ocurrencias con la misma clave se devolverá el valor de la primera. Si no existe valor para la clave especificada se devolverá la cadena `"error"`. Por ejemplo:

In [ ]:
class TestLookup(
    lookup: List[(Int, Char)] => Int => Either[String, Char]
) extends FlatSpec with Matchers{
    
    "lookup" should "work" in {
        lookup(List())(1) shouldBe Left("error")
        lookup(List((1, 'a'), (2, 'b'), (3, 'c')))(1) shouldBe Right('a')
        lookup(List((1, 'a'), (2, 'b'), (1, 'f'), (3, 'c')))(1) shouldBe Right('a')
        lookup(List((1, 'a'), (2, 'b'), (3, 'c')))(2) shouldBe Right('b')
        lookup(List((1, 'a'), (2, 'b'), (3, 'c')))(3) shouldBe Right('c')
        lookup(List((1, 'a'), (2, 'b'), (3, 'c')))(0) shouldBe Left("error")
    }
}

Implementa la función `lookup` utilizando la función `foldLeftUntil` del apartado anterior: 

In [ ]:
def lookup[A, B](list: List[(A, B)])(a: A): Either[String, B] =
    foldLeftUntil[(A, B), Either[String, B]](list)(Left("error"))(_.isRight){
        case (_, (`a`, b)) => Right(b) 
        case (acc, _) => acc 
    }

In [ ]:
run(new TestLookup(lookup))